In [84]:
import csv
import os
import re

from dataclasses import dataclass, field

@dataclass
class Lesson:
    subj: str = None
    module_num: int = 0
    lesson_num: int = 0
    url: str = None
    student_docx: str = None
    student_pdf: str = None
    teacher_docx: str = None
    teacher_pdf: str = None
    status: str = None
    notes: str = ""
    keywords: set = field(default_factory=lambda: set())

modules = []
lessons = []
subjects = set()
lsn = None
with open('../data/EngageNY/content.tsv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter='\t')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            a = row[0].strip()
            b = row[1].strip()
            c = row[2].strip()
            d = row[3].strip()
            g = row[6].strip()
            h = row[7].strip()
            i = row[8].strip()
            if len(a)>0:
                  modules.append(a)
            if len(b)>0:
                  if lsn is not None:
                      lessons.append(lsn)
                      if len(lsn.keywords) > 0:
                          print(lsn)
                  lsn = Lesson(url=b)
                  # print(f'\t{b}')
            if len(d)>0:
                  if lsn.subj is None:
                      # print(f"\t<<{d}>>")
                      subj, _ = d.split(' Module ')
                      subj = subj.replace(' Mathematics', '').strip()
                      if subj=='Precalculus and Advanced Topics':
                          subj = 'Precalculus'
                      subjects.add(subj)
                      lsn.subj = subj
                      mod_id = re.match(r".*Module ([0-9]+).*",d).group(1)
                      lsn.module_num = int(mod_id)
                      les_id = re.match(r".* Lesson ([0-9]+).*",d).group(1)
                      # print (les_id)
                      lsn.lesson_num = int (les_id)
                      # print(lsn.subj,'--',lsn.module_num,'--',lsn.lesson_num)
            if len(c)>0:
                  is_pdf = bool (re.search(r"\.pdf", c))
                  is_docx = bool (re.search(r"\.docx", c))
                  is_teacher = bool (re.search(r"-teacher\.", c))
                  if is_teacher:
                      if is_pdf:
                          lsn.teacher_pdf = c
                      if is_docx:
                          lsn.teacher_docx = c
                  else:
                      if is_pdf:
                          lsn.student_pdf = c
                      if is_docx:
                          lsn.student_docx = c
            if len(g)>0:
                  lsn.status = g.lower()
            if len(h)>0:
                  if (len(lsn.notes)>0): 
                      lsn.notes += " "
                  lsn.notes += h
            if len(i)>0:
                  lsn.keywords = lsn.keywords.union( set(re.split('[,;] ', i)) )             
                  
            line_count += 1
    if lsn is not None:
        lessons.append(lsn)
    print(f'Processed {line_count} lines, {len(modules)} modules, {len(lessons)} lessons.') 
    
    subjects
    

Column names are Module:, Lesson:, Files:, , , , Reviewed, status:, Notes:, Keywords:
Lesson(subj='Grade 6', module_num=2, lesson_num=7, url='https://www.engageny.org/resource/grade-6-mathematics-module-2-topic-lesson-7', student_docx='https://www.engageny.org/file/40456/download/math-g6-m2-topic-a-lesson-7-student.docx?token=FWHtUFlM', student_pdf='https://www.engageny.org/file/40446/download/math-g6-m2-topic-a-lesson-7-student.pdf?token=G8t55kZI', teacher_docx='https://www.engageny.org/file/40461/download/math-g6-m2-topic-a-lesson-7-teacher.docx?token=l-iyq__o', teacher_pdf='https://www.engageny.org/file/40451/download/math-g6-m2-topic-a-lesson-7-teacher.pdf?token=YWRx9qQO', status='yes', notes='', keywords={'partitive interpretation', ' measurement interpretation', 'reciprocals'})
Lesson(subj='Grade 6', module_num=2, lesson_num=10, url='https://www.engageny.org/resource/grade-6-mathematics-module-2-topic-b-lesson-10', student_docx='https://www.engageny.org/file/40531/download/math-g

{'Algebra I',
 'Algebra II',
 'Geometry',
 'Grade 1',
 'Grade 2',
 'Grade 3',
 'Grade 4',
 'Grade 5',
 'Grade 6',
 'Grade 7',
 'Grade 8',
 'Kindergarten',
 'Precalculus'}

In [2]:
from bs4 import BeautifulSoup as Bsoup
import requests
import re

std_refs = {}

def scan_engage_lesson(url):
    r  = requests.get(url)
    soup = Bsoup(r.text, 'html.parser')
    desc = None
    stds = set()
    title = soup.title.text
    div_w_sum = soup.find_all('div', {"class": "field-type-text-with-summary"})
    for d in div_w_sum:
        desc = d.text.strip()
        break
        
    std_ref = re.compile("^([A -Z0-9a-d]+\.){2,5}[A-Z0-9a-d]+$")        
    links = soup.find_all('a')
    for l in links:
        t = l.text.strip()
        if std_ref.search(t):
            stds.add(t);
            std_refs[t] = f'https://www.engageny.org{l["href"]}'
    return title,desc,stds
                  
                  
for i,url in enumerate(lessons):
    if i%11 !=7 :
        continue;

    title,desc,stds = scan_engage_lesson(url)
    print(url)
    print(title)
    print(desc)
    print(', '.join(stds))            
                         
    print('= = = = = =')
    continue
    

    print(url)
    r  = requests.get(url)

    soup = Bsoup(r.text, 'html.parser')

    print(soup.title.text)

    div_w_sum = soup.find_all('div', {"class": "field-type-text-with-summary"})
    for d in div_w_sum:
        print('-------')
        print(d.text)
        print('-------')
        for li in d.find_all('li'):
            print(li.text)

    outcomes = re.compile("^Student Outcome")

    print('-------')
    divs = soup.find_all('div')
    for d in divs:
        if len(d.find_all('div')) > 0:
            continue
        t = d.text
        if outcomes.search(t):
            print(t)
                
        
    std_ref = re.compile("^([A -Z0-9a-d]+\.){2,5}[A-Z0-9a-d]+$")        

    links = soup.find_all('a')

    for l in links:
        t = l.text.strip()
        if std_ref.search(t):
            print(t)
            print(f'https://www.engageny.org{l["href"]}')



https://www.engageny.org/resource/kindergarten-mathematics-module-1-topic-c-lesson-8
Kindergarten Mathematics Module 1, Topic C, Lesson 8 | EngageNY
Objective: Answer how many questions to 5 in linear configurations (5-group) with 4 in an array configuration. Compare ways to count 5 fingers.
K.CC.6
= = = = = =
https://www.engageny.org/resource/kindergarten-mathematics-module-1-topic-e-lesson-19
Kindergarten Mathematics Module 1, Topic E, Lesson 19 | EngageNY
Objective: Count 5─7 linking cubes in linear configurations. Match with numeral 7. Count on fingers from 1 to 7 and connect to 5-group images.
K.CC.4.a, K.CC.3, K.CC.5, K.CC.4.b
= = = = = =
https://www.engageny.org/resource/kindergarten-mathematics-module-1-topic-g-lesson-30
Kindergarten Mathematics Module 1, Topic G, Lesson 30 | EngageNY
Objective: Exploration: Make math stairs from 1 to 10 in cooperative groups.
K.CC.4.a, K.CC.5, K.CC.4.b, K.CC.2, K.CC.4.c
= = = = = =
https://www.engageny.org/resource/kindergarten-mathematics-mod

https://www.engageny.org/resource/algebra-ii-module-1-topic-d-lesson-37
Algebra II Module 1, Topic D, Lesson 37 | EngageNY
Student Outcomes 
Students define a complex number in the form a + bi, where a and b are real numbers and the imaginary unit i satisfies i 2 = −1.  Students geometrically identify i as a multiplicand effecting a 90° counterclockwise rotation of the real number line. Students locate points corresponding to complex numbers in the complex plane.
Students understand complex numbers as a superset of the real numbers; i.e., a complex number a + bi is real when b  = 0. Students learn that complex numbers share many similar properties of the real numbers: associative, commutative, distributive, addition/subtraction, multiplication, etc.
N.CN.2, N.CN.1
= = = = = =
https://www.engageny.org/resource/algebra-ii-module-2-topic-a-lesson-8
Algebra II Module 2, Topic A, Lesson 8 | EngageNY
Student Outcomes
Students graph the sine and cosine functions and analyze the shape of these

https://www.engageny.org/resource/precalculus-and-advanced-topics-module-2-topic-a-lesson-3
Precalculus and Advanced Topics Module 2, Topic A, Lesson 3 | EngageNY
Student Outcomes
Students use matrices to represent data based on transportation networks.
Students multiply matrices of appropriate dimensions and interpret the meaning of this arithmetic in terms of transportation networks.
N.VM.8, N.VM.7
= = = = = =
https://www.engageny.org/resource/precalculus-and-advanced-topics-module-2-topic-c-lesson-14
Precalculus and Advanced Topics Module 2, Topic C, Lesson 14 | EngageNY
Student Outcomes
Students will represent systems of equations as linear transformations of the form Lx = b, using matrix notation.
Students will discover that the systems of equations written in the form Lx = b can be solved by computing x = L-1b for all invertible matrices, L and they will apply this process to find solutions to systems of two linear equations in two variables.
A.REI.9, A.REI.8, N.VM.10
= = = = = =

https://www.engageny.org/resource/grade-2-mathematics-module-4-topic-f-lesson-30
Grade 2 Mathematics Module 4, Topic F, Lesson 30 | EngageNY
Objective: Compare totals below to new groups below as written methods.
2.NBT.9, 2.NBT.7
= = = = = =
https://www.engageny.org/resource/grade-2-mathematics-module-5-topic-b-lesson-10
Grade 2 Mathematics Module 5, Topic B, Lesson 10 | EngageNY
Objective: Use math drawings to represent additions with up to two compositions and relate drawings to the addition algorithm.
2.NBT.9, 2.NBT.7
= = = = = =
https://www.engageny.org/resource/grade-2-mathematics-module-6-topic-lesson-1
Grade 2 Mathematics Module 6, Topic A, Lesson 1 | EngageNY
Objective: Use manipulatives to create equal groups.
2.OA.4
= = = = = =
https://www.engageny.org/resource/grade-2-mathematics-module-6-topic-c-lesson-12
Grade 2 Mathematics Module 6, Topic C, Lesson 12 | EngageNY
Objective: Use math drawings to compose a rectangle with square tiles.
2.G.2
= = = = = =
https://www.engageny.o

https://www.engageny.org/resource/grade-4-mathematics-module-5-topic-g-lesson-39
Grade 4 Mathematics Module 5, Topic G, Lesson 39 | EngageNY
Objective:  Solve multiplicative comparison word problems involving fractions.
4.OA.2
= = = = = =
https://www.engageny.org/resource/grade-4-mathematics-module-6-topic-c-lesson-9
Grade 4 Mathematics Module 6, Topic C, Lesson 9 | EngageNY
Objective:  Use the place value chart and metric measurement to compare decimals and answer comparison questions.
4.NF.7
= = = = = =
https://www.engageny.org/resource/grade-4-mathematics-module-7-topic-lesson-4
 Grade 4 Mathematics Module 7, Topic A, Lesson 4 | EngageNY
Objective:  Solve multiplicative comparison word problems using measurement conversion tables.
4.OA.2, 4.OA.1
= = = = = =
https://www.engageny.org/resource/grade-4-mathematics-module-7-topic-d-lesson-15
Grade 4 Mathematics Module 7, Topic D, Lesson 15 | EngageNY
Objective:  Create and determine the area of composite figures.
4.MD.3
= = = = = =
https

https://www.engageny.org/resource/grade-6-mathematics-module-4-topic-h-lesson-34
Grade 6 Mathematics Module 4, Topic H, Lesson 34 | EngageNY
Student Outcomes
Students recognize that inequalities of the form x < c and x > c, where x is a variable and c is a fixed number have infinitely many solutions when the values of x come from a set of rational numbers.
6.EE.5, 6.EE.8
= = = = = =
https://www.engageny.org/resource/grade-6-mathematics-module-5-topic-c-lesson-11
Grade 6 Mathematics Module 5, Topic C, Lesson 11 | EngageNY
Student Outcomes 
Students extend their understanding of the volume of a right rectangular prism with integer side lengths to right rectangular prisms with fractional side lengths.  They apply the formula V = lwh to find the volume of a right rectangular prism and use the correct volume units when writing the answer.
6.G.2
= = = = = =
https://www.engageny.org/resource/grade-6-mathematics-module-6-topic-lesson-3
Grade 6 Mathematics Module 6, Topic A, Lesson 3 | EngageNY

https://www.engageny.org/resource/grade-8-mathematics-module-3-topic-lesson-1
Grade 8 Mathematics Module 3, Topic A, Lesson 1 | EngageNY
Student Outcomes
Students learn the definition of dilation and why “same shape” is not good enough to say two figures are similar.
Students know that dilations magnify and shrink figures.
8.G.3
= = = = = =
https://www.engageny.org/resource/grade-8-mathematics-module-3-topic-b-lesson-12
Grade 8 Mathematics Module 3, Topic B, Lesson 12 | EngageNY
Student Outcomes
Students use properties of similar triangles to solve real-world problems.
8.G.4
= = = = = =
https://www.engageny.org/resource/grade-8-mathematics-module-4-topic-lesson-9
Grade 8 Mathematics Module 4, Topic A, Lesson 9 | EngageNY
Student Outcomes
Students know how to rewrite an exponential expression that represents a series as a linear equation.
8.EE.7.a
= = = = = =
https://www.engageny.org/resource/grade-8-mathematics-module-4-topic-c-lesson-20
Grade 8 Mathematics Module 4, Topic C, Lesson 20

In [30]:
import spacy

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")

def scan_engage_std(url):
    r  = requests.get(url)
    soup = Bsoup(r.text, 'html.parser')
    desc = None
    stds = set()
    title = soup.title.text
    dl = soup.find('dl')
    dt = dl.find_all('dt')
    dd = dl.find_all('dd')
    desc = {}
    for i in range(len(dt)):
        key = dt[i].text.strip()
        key = re.sub(r"[:.]$", "", key)
        if not key in ['Category', 'State Standard', 'Sub-Category']:
            raise (f"Unexpected key {key}")
        desc[key] = dd[i].text.strip()
    return desc

def nlp_v_np(txt):
    doc = nlp(txt)
    nc = [chunk.text for chunk in doc.noun_chunks])
    v = [[token.lemma_ for token in doc if token.pos_ == "VERB"]]
    
    
    print("Noun phrases:\n\t-", '\n\t- '.join())
    print("Verbs:\n\t-", '\n\t- '.join())


i = 0
for s in std_refs:
    i+=1
    if i%7!=3:
        continue;
    url = std_refs[s]
    print(s,url)
    desc = scan_engage_std(url)
    # print(desc)
    doc = nlp(desc['State Standard'])
    print("Noun phrases:\n\t-", '\n\t- '.join([chunk.text for chunk in doc.noun_chunks]))
    print("Verbs:\n\t-", '\n\t- '.join([token.lemma_ for token in doc if token.pos_ == "VERB"]))



K.CC.4.a https://www.engageny.org/ccls-math/kcc4a
Noun phrases:
	- objects
	- the number names
	- the standard order
	- each object
	- only one number name
	- one and only one object
Verbs:
	- count
	- say
	- pair
K.MD.2 https://www.engageny.org/ccls-math/kmd2
Noun phrases:
	- two objects
	- a measurable attribute
	- which object
	- “more of”/“less
	- the attribute
	- the difference
	- example
	- the heights
	- two children
	- one child
	- taller/shorter
Verbs:
	- compare
	- see
	- have
	- describe
	- compare
	- describe
K.G.6 https://www.engageny.org/ccls-math/kg6
Noun phrases:
	- simple shapes
	- larger shapes
	- example
	- you
	- these two triangles
	- full sides
	- a rectangle
Verbs:
	- form
	- Can
	- join
	- touch
	- make
S.ID.6.c https://www.engageny.org/ccls-math/sid6c
Noun phrases:
	- a linear function
	- a scatter plot
	- a linear association
Verbs:
	- fit
	- suggest
A.SSE.2 https://www.engageny.org/ccls-math/asse2
Noun phrases:
	- the structure
	- an expression
	- ways
	- it
